In [2]:
import yfinance as yf
import pandas as pd
from yfinance import EquityQuery
from collections import OrderedDict

In [3]:
def rolling_calendar_returns(data, days):
    
    # Calculate returns
    returns = data.pct_change(periods=days)
    
    return returns

In [4]:
def strict_elementwise_average(*dfs):
    """
    Calculate element-wise average of multiple DataFrames, returning NaN if any value is missing
    
    Parameters:
    *dfs : variable number of pandas DataFrames with identical shapes
    
    Returns:
    pandas DataFrame with strict averaging (NaN if any input is NaN)
    """
    if not dfs:
        raise ValueError("At least one DataFrame must be provided")
    
    # Initialize with first DataFrame
    sum_df = dfs[0].copy()
    count_df = (~dfs[0].isna()).astype(int)
    
    # Accumulate sum and count
    for df in dfs[1:]:
        sum_df = sum_df.add(df, fill_value=0)
        count_df = count_df.add(~df.isna(), fill_value=0)
    
    # Calculate average only where count equals number of DataFrames
    total_dfs = len(dfs)
    avg_df = sum_df.where(count_df == total_dfs) / total_dfs
    return avg_df

In [5]:
def get_volume_matrix(tickers):
    """Return DataFrame with dates as index, tickers as columns, and volumes as values"""
    return yf.download(tickers, period="max", progress=False)["Volume"]

In [6]:
def get_price_matrix(tickers, period="max"):
    """Downloads historical prices, skips failed tickers, and returns DataFrame.
    
    Args:
        tickers: List of symbols
        period: Time period to download (default: max)
    
    Returns:
        DataFrame with dates as index, successful tickers as columns
    """
    successful_data = []
    for ticker in tickers:
        try:
            data = yf.download(ticker, period=period, progress=False)["Close"]
            data.name = ticker  # Set column name to ticker
            successful_data.append(data)
        except Exception as e:
            print(f"✗ Failed {ticker}: {str(e).split('(')[0]}")  # Concise error
    
    return pd.concat(successful_data, axis=1)  # Combine all successful downloads

# Usage:
# ETFs_daily_prices = download_prices(all_etfs_list)

In [7]:
def extract_stock_list(sector_input):
    # Define the custom query
    custom_query = EquityQuery('and', [
        EquityQuery('eq', ['region', 'us']),
        EquityQuery('gte', ['dayvolume', 10000]),
        EquityQuery('gte', ['intradayprice', 5]),
         EquityQuery('eq', ['sector', sector_input]) # Exclude penny stocks
    ])

    response = yf.screen(custom_query, size = 250, sortField = 'intradaymarketcap', sortAsc = False)

    # Extract relevant fields from the response
    quotes = response['quotes']
    results = [
        {
            'ticker': stock['symbol'],
        }
        for stock in quotes
    ]

    # Extract tickers as a list
    tickers = [stock['symbol'] for stock in response['quotes']]
    print(tickers)
    
    return tickers

# Usage:
extract_stock_list('Healthcare')

['LLY', 'UNH', 'JNJ', 'ABBV', 'NVO', 'RHHBY', 'ABT', 'AZN', 'MRK', 'NVS', 'TMO', 'ISRG', 'AMGN', 'BSX', 'DHR', 'PFE', 'SYK', 'GILD', 'SNY', 'VRTX', 'BMY', 'MDT', 'ELV', 'CI', 'HCA', 'MCK', 'CVS', 'GSK', 'CSLLY', 'CHGCY', 'ZTS', 'REGN', 'BDX', 'MKKGY', 'COR', 'TAK', 'ALC', 'HLN', 'DSNKY', 'EW', 'VEEV', 'GEHC', 'ARGX', 'ALNY', 'IDXX', 'A', 'CAH', 'RMD', 'IQV', 'HUM', 'CNC', 'DXCM', 'MTD', 'CLPBY', 'PHG', 'BAYRY', 'ZBH', 'STE', 'WAT', 'BNTX', 'BIIB', 'LH', 'DGX', 'MOH', 'NTRA', 'PODD', 'RPRX', 'BAX', 'TEVA', 'COO', 'WST', 'OLYMY', 'FMS', 'ICLR', 'SMMT', 'HOLX', 'ITCI', 'UTHR', 'INSM', 'BMRN', 'SOLV', 'RVTY', 'THC', 'ILMN', 'DVA', 'GMAB', 'UHS', 'SNN', 'CORT', 'ALGN', 'SSMXY', 'INCY', 'NBIX', 'RDY', 'AVTR', 'MRNA', 'DOCS', 'EXEL', 'VTRS', 'PEN', 'EHC', 'GMED', 'ATR', 'WBA', 'MEDP', 'TECH', 'ASND', 'CHE', 'QGEN', 'MASI', 'HSIC', 'HALO', 'EXAS', 'HQY', 'CRL', 'TEM', 'JAZZ', 'ENSG', 'ROIV', 'MDGL', 'RGEN', 'BIO', 'LNTH', 'BBIO', 'WAY', 'TFX', 'BRKR', 'RVMD', 'LEGN', 'MMSI', 'TGTX', 'SRPT', 'O

['LLY',
 'UNH',
 'JNJ',
 'ABBV',
 'NVO',
 'RHHBY',
 'ABT',
 'AZN',
 'MRK',
 'NVS',
 'TMO',
 'ISRG',
 'AMGN',
 'BSX',
 'DHR',
 'PFE',
 'SYK',
 'GILD',
 'SNY',
 'VRTX',
 'BMY',
 'MDT',
 'ELV',
 'CI',
 'HCA',
 'MCK',
 'CVS',
 'GSK',
 'CSLLY',
 'CHGCY',
 'ZTS',
 'REGN',
 'BDX',
 'MKKGY',
 'COR',
 'TAK',
 'ALC',
 'HLN',
 'DSNKY',
 'EW',
 'VEEV',
 'GEHC',
 'ARGX',
 'ALNY',
 'IDXX',
 'A',
 'CAH',
 'RMD',
 'IQV',
 'HUM',
 'CNC',
 'DXCM',
 'MTD',
 'CLPBY',
 'PHG',
 'BAYRY',
 'ZBH',
 'STE',
 'WAT',
 'BNTX',
 'BIIB',
 'LH',
 'DGX',
 'MOH',
 'NTRA',
 'PODD',
 'RPRX',
 'BAX',
 'TEVA',
 'COO',
 'WST',
 'OLYMY',
 'FMS',
 'ICLR',
 'SMMT',
 'HOLX',
 'ITCI',
 'UTHR',
 'INSM',
 'BMRN',
 'SOLV',
 'RVTY',
 'THC',
 'ILMN',
 'DVA',
 'GMAB',
 'UHS',
 'SNN',
 'CORT',
 'ALGN',
 'SSMXY',
 'INCY',
 'NBIX',
 'RDY',
 'AVTR',
 'MRNA',
 'DOCS',
 'EXEL',
 'VTRS',
 'PEN',
 'EHC',
 'GMED',
 'ATR',
 'WBA',
 'MEDP',
 'TECH',
 'ASND',
 'CHE',
 'QGEN',
 'MASI',
 'HSIC',
 'HALO',
 'EXAS',
 'HQY',
 'CRL',
 'TEM',
 'JAZZ',
 'E

In [8]:
def filter_liquid(tickers, min_turnover = 5e6):
    """Filters tickers by 20-day average dollar turnover (Volume * Close)."""
    data = yf.download(tickers, period="20d", progress=False)
    liquid = (data['Volume'] * data['Close']).mean() >= min_turnover
    return liquid[liquid].index.tolist()

# Usage:
liquid_stocks = filter_liquid(extract_stock_list('Healthcare'))

['LLY', 'UNH', 'JNJ', 'ABBV', 'NVO', 'RHHBY', 'ABT', 'AZN', 'MRK', 'NVS', 'TMO', 'ISRG', 'AMGN', 'BSX', 'DHR', 'PFE', 'SYK', 'GILD', 'SNY', 'VRTX', 'BMY', 'MDT', 'ELV', 'CI', 'HCA', 'MCK', 'CVS', 'GSK', 'CSLLY', 'CHGCY', 'ZTS', 'REGN', 'BDX', 'MKKGY', 'COR', 'TAK', 'ALC', 'HLN', 'DSNKY', 'EW', 'VEEV', 'GEHC', 'ARGX', 'ALNY', 'IDXX', 'A', 'CAH', 'RMD', 'IQV', 'HUM', 'CNC', 'DXCM', 'MTD', 'CLPBY', 'PHG', 'BAYRY', 'ZBH', 'STE', 'WAT', 'BNTX', 'BIIB', 'LH', 'DGX', 'MOH', 'NTRA', 'PODD', 'RPRX', 'BAX', 'TEVA', 'COO', 'WST', 'OLYMY', 'FMS', 'ICLR', 'SMMT', 'HOLX', 'ITCI', 'UTHR', 'INSM', 'BMRN', 'SOLV', 'RVTY', 'THC', 'ILMN', 'DVA', 'GMAB', 'UHS', 'SNN', 'CORT', 'ALGN', 'SSMXY', 'INCY', 'NBIX', 'RDY', 'AVTR', 'MRNA', 'DOCS', 'EXEL', 'VTRS', 'PEN', 'EHC', 'GMED', 'ATR', 'WBA', 'MEDP', 'TECH', 'ASND', 'CHE', 'QGEN', 'MASI', 'HSIC', 'HALO', 'EXAS', 'HQY', 'CRL', 'TEM', 'JAZZ', 'ENSG', 'ROIV', 'MDGL', 'RGEN', 'BIO', 'LNTH', 'BBIO', 'WAY', 'TFX', 'BRKR', 'RVMD', 'LEGN', 'MMSI', 'TGTX', 'SRPT', 'O

In [ ]:
def get_correlation_matrix_and_returns(tickers, determine_correlation_period = "5Y", determine_return_period = "1Y"):
    """Fetch historical data and calculate correlation matrix and 1-year returns.
    
    Args:
        tickers (list): List of stock/ETF tickers to analyze
        
    Returns:
        tuple: (correlation_matrix, one_year_returns) 
               - correlation_matrix: DataFrame of pairwise correlations
               - one_year_returns: Series of 1-year returns for each ticker
    """
    
    # Download historical data to calcuate correlation
    price_for_correlation = get_price_matrix(tickers, period = determine_correlation_period)
    returns_correlation = price_for_correlation.pct_change().dropna(how = "all")
    corr_matrix = returns_correlation.corr()
    print(f'Price Matrix:\n {price_for_correlation}')
    print(f'Correlation Matrix:\n {corr_matrix}')
    
    # Download historical data to calcuate returns, drop ticker with lower returns if two tickers are highly correlated
    price_for_return = get_price_matrix(tickers, period = determine_return_period)
    n_year_returns = (price_for_return.iloc[-1] / price_for_return.iloc[0] - 1).dropna()

    # Calculate daily returns and correlation matrix


    # Calculate 1-year returns for each ticker
    # data_1y = returns_correlation.loc[start_date_1y:end_date]
    
    return corr_matrix, n_year_returns

def filter_high_correlation_tickers(tickers, max_correlation=0.3):
    """Filter out highly correlated tickers, keeping the best performer.
    
    Args:
        tickers (list): List of tickers to filter
        max_correlation (float): Threshold for considering correlation too high
        
    Returns:
        list: Filtered list of tickers with correlations below threshold
    """
    current_tickers = tickers.copy()
    removed_tickers = set()
    
    while True:
        # Get current tickers that are still present and not removed
        active_tickers = [t for t in current_tickers if t not in removed_tickers]
        if len(active_tickers) <= 1:
            break
        
        try:
            corr_matrix, one_year_returns = get_correlation_matrix_and_returns(active_tickers)
        except Exception as e:
            print(f"Error fetching data: {e}")
            break
        
        # Find pairs with correlation >= max_correlation
        high_corr_pairs = []
        n = len(active_tickers)
        for i in range(n):
            for j in range(i+1, n):
                ticker_i = active_tickers[i]
                ticker_j = active_tickers[j]
                corr = corr_matrix.loc[ticker_i, ticker_j]
                if abs(corr) >= max_correlation:
                    high_corr_pairs.append((ticker_i, ticker_j, corr))
        
        if not high_corr_pairs:
            break
        
        # Determine which tickers to remove
        to_remove = set()
        for pair in high_corr_pairs:
            ticker_i, ticker_j, _ = pair
            ret_i = one_year_returns.get(ticker_i, -np.inf)
            ret_j = one_year_returns.get(ticker_j, -np.inf)
            
            if ret_i < ret_j:
                to_remove.add(ticker_i)
            else:
                to_remove.add(ticker_j)
        
        if not to_remove:
            break
        
        removed_tickers.update(to_remove)
    
    filtered_tickers = [t for t in current_tickers if t not in removed_tickers]
    return filtered_tickers

# Example usage
# filtered_tickers = filter_high_correlation_tickers(all_etfs_list, 0.9)
# print(f'Number of tickers with low correlation: {len(filtered_tickers)}')
# print(f'Tickers with low correlation: {filtered_tickers}')


1 Failed download:
['BZF']: YFPricesMissingError('possibly delisted; no price data found  (period=5y)')

1 Failed download:
['EMFX']: YFPricesMissingError('possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['VXXB']: YFPricesMissingError('possibly delisted; no price data found  (period=5y)')

1 Failed download:
['FOIL']: YFPricesMissingError('possibly delisted; no price data found  (period=5y)')

1 Failed download:
['PSCL']: YFPricesMissingError('possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['BITE']: YFPricesMissingError('possibly delisted; no price data found  (period=5y) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['IXIS']: YFPricesMissingError('possibly delisted; no price data found  (period=5y)')


Price Matrix:
 Ticker            SHY       VGSH       SCHO        BIL        GBIL  \
Date                                                                 
2020-04-01  78.671242  55.615219  21.885933  81.236351   89.075668   
2020-04-02  78.689407  55.606289  21.873232  81.245239   89.075668   
2020-04-03  78.671242  55.588413  21.877462  81.218620   89.084526   
2020-04-06  78.616760  55.552681  21.856291  81.227478   89.057976   
2020-04-07  78.598595  55.552681  21.864758  81.209747   89.004883   
...               ...        ...        ...        ...         ...   
2025-03-25  82.500000  58.549999  24.290001  91.669998  100.110001   
2025-03-26  82.480003  58.540001  24.280001  91.680000  100.120003   
2025-03-27  82.519997  58.560001  24.299999  91.699997  100.129997   
2025-03-28  82.669998  58.680000  24.330000  91.730003  100.169998   
2025-03-31  82.730003  58.689999  24.340000  91.730003  100.180000   

Ticker             IEI         IEF       VGIT       GOVT       SPTI  ...  


1 Failed download:
['SNLN']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['JJM']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['BZF']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['FXCH']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['EMFX']: YFPricesMissingError('possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['VXXB']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['JJU']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['FOIL']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['JJC']: YFPricesMissingError('possibly delisted; no price data found  (period=

Price Matrix:
 Ticker           SCHO        BIL        GBIL       SPTI       VTIP       SCHP  \
Date                                                                            
2020-04-01  21.885927  81.236366   89.075706  30.053680  40.721409  21.981033   
2020-04-02  21.873224  81.245232   89.075706  30.026468  41.122440  22.282047   
2020-04-03  21.877468  81.218613   89.084541  30.026468  41.322948  22.449696   
2020-04-06  21.856281  81.227486   89.057983  29.935722  41.222702  22.411587   
2020-04-07  21.864759  81.209732   89.004898  29.899416  41.247757  22.438255   
...               ...        ...         ...        ...        ...        ...   
2025-03-25  24.290001  91.669998  100.110001  28.410000  49.619999  26.620001   
2025-03-26  24.280001  91.680000  100.120003  28.360001  49.619999  26.580000   
2025-03-27  24.299999  91.699997  100.129997  28.370001  49.709999  26.629999   
2025-03-28  24.330000  91.730003  100.169998  28.520000  49.860001  26.780001   
2025-03-31  2


1 Failed download:
['SNLN']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['JJM']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['BZF']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['FXCH']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['EMFX']: YFPricesMissingError('possibly delisted; no price data found  (period=1y) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['VXXB']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['JJU']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['FOIL']: YFPricesMissingError('possibly delisted; no price data found  (period=1y)')

1 Failed download:
['JJC']: YFPricesMissingError('possibly delisted; no price data found  (period=

Number of tickers with low correlation: 110


"Tickers with low correlation: ['SCHO', 'BIL', 'GBIL', 'SPTI', 'VTIP', 'SCHP', 'IGIB', 'HYLB', 'BKLN', 'SRLN', 'SNLN', 'FLBL', 'BWX', 'BNDX', 'EMLC', 'HYEM', 'EMHY', 'VTEB', 'TFI', 'MUNI', 'HYMB', 'SHYD', 'HYD', 'SGOL', 'USO', 'UNG', 'DBE', 'DBA', 'CORN', 'WEAT', 'SOYB', 'CANE', 'DBB', 'JJM', 'COPX', 'REMX', 'UUP', 'FXY', 'FXB', 'CEW', 'BZF', 'FXCH', 'EMFX', 'VIXY', 'SVOL', 'VXXB', 'ETHE', 'ARKB', 'RPAR', 'DIVB', 'AOR', 'VEGI', 'XHB', 'VAW', 'DOW', 'CUT', 'FUND', 'JJU', 'FOIL', 'JJC', 'GDX', 'SIVR', 'PPLT', 'PALL', 'KOL', 'SLX', 'PSCL', 'CARZ', 'PBS', 'XRT', 'EMFM', 'PKG', 'PEJ', 'BITE', 'RTH', 'LUXE', 'ONLN', 'FDIS', 'BETZ', 'JETS', 'TRYP', 'IAI', 'KRE', 'IXIS', 'FINX', 'IAK', 'REZ', 'SRET', 'PBJ', 'FTXG', 'XLP', 'VICE', 'IBB', 'PJP', 'XPH', 'IHF', 'IHI', 'XHE', 'QCLN', 'ICLN', 'XLU', 'XLE', 'AMLP', 'CRAK', 'URA', 'ITA', 'IYT', 'SEA', 'IGV', 'SMH']"

In [9]:
sector_etfs = {
    # Materials & Commodities
    "Agricultural_Inputs": ["VEGI", "MOO"],
    "Building_Materials": ["XHB", "ITB"],
    "Chemicals": ["VAW", "DOW"],
    "Specialty_Chemicals": ["FXZ"],
    "Lumber_Wood_Production": ["WOOD", "CUT"],
    "Paper_Products": ["FUND"],  # Part of broader materials ETFs
    "Aluminum": ["JJU", "FOIL"],
    "Copper": ["COPX", "JJC"],
    "Industrial_Metals_Mining": ["PICK", "REMX"],
    "Gold": ["GLD", "IAU", "GDX"], # Miners
    "Silver": ["SLV", "SIVR", "SIL"], # Miners
    "Precious_Metals_Mining": ["PPLT", "PALL", "SILJ"],
    "Coking_Coal": ["KOL"],  # Global coal ETF
    "Steel": ["SLX", "PSCL"],

    # Consumer Discretionary
    "Auto_Dealerships": ["CARZ"],
    "Auto_Manufacturers": ["CARZ", "VCR"],
    "Auto_Parts": ["CARZ"],
    "Recreational_Vehicles": ["PBS"],  # Subset of consumer cyclicals
    "Furnishings_Fixtures_Appliances": ["XHB", "PBS"],
    "Residential_Construction": ["ITB", "XHB"],
    "Textile_Apparel_Footwear": ["XRT", "EMFM"], # retail-heavy, emerging markets focus
    "Packaging_Containers": ["PKG", "XAR"],
    "Personal_Services": ["PEJ"],
    "Restaurants": ["BITE", "PEJ"],
    
    # Retail
    "Apparel_Retail": ["XRT", "RTH"],
    "Department_Stores": ["XRT", "RTH"],
    "Home_Improvement_Retail": ["XHB", "ITB"],
    "Luxury_Goods": ["LUXE", "ONLN"],
    "Internet_Retail": ["ONLN", "IBUY"],
    "Specialty_Retail": ["XRT", "FDIS"],
    
    # Leisure & Travel
    "Gambling": ["BETZ"],
    "Leisure": ["PEJ"],
    "Lodging": ["PEJ"],
    "Resorts_Casinos": ["BETZ"],
    "Travel_Services": ["JETS", "TRYP"],
    
    # Financials
    "Asset_Management": ["KCE", "IAI"],
    "Banks_Diversified": ["KBE"],
    "Banks_Regional": ["KRE", "QABA"],
    "Mortgage_Finance": ["REM", "MORT"],
    "Capital_Markets": ["KCE", "IAI"],
    "Financial_Data_Exchanges": ["IXIS", "FINX"],
    
    # Insurance
    "Insurance_Life": ["KIE", "IAK"],
    "Insurance_Property_Casualty": ["KIE", "IAK"],
    "Insurance_Reinsurance": ["KIE", "IAK"],
    "Insurance_Specialty": ["KIE", "IAK"],
    "Insurance_Brokers": ["KIE", "IAK"],
    "Insurance_Diversified": ["KIE", "IAK"],
    
    # Real Estate
    "Real_Estate_Development": ["REZ"],
    "Real_Estate_Services": ["REZ"],
    "REIT_Healthcare": ["REZ"],
    "REIT_Hotel_Motel": ["REZ"],
    "REIT_Industrial": ["REZ"],
    "REIT_Office": ["REZ"],
    "REIT_Residential": ["REZ"],
    "REIT_Retail": ["REZ"],
    "REIT_Mortgage": ["REM"],
    "REIT_Specialty": ["SRET"],
    "REIT_Diversified": ["VNQ", "XLRE"],
    
    # Consumer Staples
    "Beverages_Brewers": ["BIB"],
    "Beverages_Wineries_Distilleries": ["BIB"],
    "Beverages_Non_Alcoholic": ["PBJ"],
    "Confectioners": ["PBJ", "FTXG"],
    "Farm_Products": ["DBA", "VEGI"],
    "Household_Personal_Products": ["XLP"],
    "Tobacco": ["XLP", "VICE"],
    
    # Healthcare
    "Biotechnology": ["IBB", "XBI"],
    "Drug_Manufacturers_General": ["PJP", "XPH"],
    "Drug_Manufacturers_Specialty": ["PJP", "XPH"],
    "Healthcare_Plans": ["IHF"],
    "Medical_Care_Facilities": ["IHF"],
    "Medical_Devices": ["IHI", "XHE"],
    "Pharmaceutical_Retailers": ["XPH"],
    
    # Utilities
    "Utilities_Renewable": ["QCLN", "ICLN"],
    "Utilities_Regulated": ["XLU", "VPU"],
    
    # Energy
    "Oil_Gas_Drilling": ["XOP"],
    "Oil_Gas_EP": ["XOP"],
    "Oil_Gas_Integrated": ["XLE"],
    "Oil_Gas_Midstream": ["AMLP"],
    "Oil_Gas_Refining": ["CRAK"],
    "Thermal_Coal": ["KOL"],
    "Uranium": ["URA", "URNM"],
    
    # Industrials
    "Aerospace_Defense": ["ITA", "PPA"],
    "Industrial_Machinery": ["XAR"],
    "Railroads": ["XTN", "IYT"],
    "Shipping": ["SEA"],
    
    # Technology
    "Software": ["IGV", "WCLD"],
    "Semiconductors": ["SOXX", "SMH"],
    "Solar": ["TAN", "ICLN"]
}

# Example: Access all Biotech ETFs
biotech_etfs = sector_etfs["Biotechnology"]
print(f"Biotech ETFs: {biotech_etfs}")

# Example: Download all Gold-related ETFs
gold_etfs = sector_etfs["Gold"]
print(f"Gold ETFs: {gold_etfs}")

Biotech ETFs: ['IBB', 'XBI']
Gold ETFs: ['GLD', 'IAU', 'GDX']


In [10]:
ficc_etfs = {
    # ======== U.S. TREASURIES ========
    "Treasuries": {
        "Short_Term": ["SHY", "VGSH", "SCHO", "BIL", "GBIL"],
        "Intermediate": ["IEI", "IEF", "VGIT", "GOVT", "SPTI"],
        "Long_Term": ["TLT", "VGLT", "SPTL", "EDV", "TMF"],
        "TIPS": ["TIP", "VTIP", "SCHP", "LTPZ", "STIP"]
    },
    
    # ======== CORPORATE BONDS ========
    "Corporate_Bonds": {
        "Investment_Grade": ["LQD", "VCIT", "VCSH", "IGIB", "IGLB"],
        "High_Yield": ["HYG", "JNK", "HYLB", "ANGL", "HYBB"],
        "Bank_Loans": ["BKLN", "SRLN", "SNLN", "FLBL"]
    },
    
    # ======== INTERNATIONAL BONDS ========
    "International_Bonds": {
        "Developed_Markets": ["BWX", "IGOV", "BNDX", "IGOV"],
        "Emerging_Markets": ["EMB", "PCY", "EMLC", "HYEM", "EMHY"]
    },
    
    # ======== MUNICIPAL BONDS ========
    "Municipal_Bonds": {
        "National": ["MUB", "VTEB", "TFI", "MUNI"],
        "High_Yield_Munis": ["HYMB", "SHYD", "HYD"]
    },
    
    # ======== COMMODITIES ======== 
    "Commodities": {
        "Precious_Metals": ["GLD", "IAU", "SLV", "GLTR", "SGOL"],
        "Energy": ["USO", "BNO", "UNG", "UCO", "DBE"],
        "Agriculture": ["DBA", "CORN", "WEAT", "SOYB", "CANE"],
        "Industrial_Metals": ["DBB", "JJM", "COPX", "REMX"]
    },
    
    # ======== CURRENCIES ========
    "Currencies": {
        "Major_FX": ["UUP", "UDN", "FXE", "FXY", "FXB"],
        "Emerging_FX": ["CEW", "BZF", "FXCH", "EMFX"]
    },
    
    # ======== SPECIALTY FICC ========
    "Specialty": {
        "Volatility": ["VIXY", "SVOL", "VXXB"],  # VXX replaced with VXXB
        "Crypto": ["BITO", "ETHE", "BTF", "ARKB"],
        "Multi_Asset": ["RPAR", "GTIP", "DIVB", "AOR"]
    }
}

# Example: Access all Treasury ETFs
treasury_etfs = [etf for sublist in ficc_etfs["Treasuries"].values() for etf in sublist]
print(f"All Treasury ETFs: {treasury_etfs}")

# Example: Download all commodity ETFs
commodity_etfs = [etf for sublist in ficc_etfs["Commodities"].values() for etf in sublist]
print(f"Commodity ETFs: {commodity_etfs}")

All Treasury ETFs: ['SHY', 'VGSH', 'SCHO', 'BIL', 'GBIL', 'IEI', 'IEF', 'VGIT', 'GOVT', 'SPTI', 'TLT', 'VGLT', 'SPTL', 'EDV', 'TMF', 'TIP', 'VTIP', 'SCHP', 'LTPZ', 'STIP']
Commodity ETFs: ['GLD', 'IAU', 'SLV', 'GLTR', 'SGOL', 'USO', 'BNO', 'UNG', 'UCO', 'DBE', 'DBA', 'CORN', 'WEAT', 'SOYB', 'CANE', 'DBB', 'JJM', 'COPX', 'REMX']


In [11]:
# Simple flattening with list comprehension
sector_etfs_list = [
    etf 
    for subcategory in sector_etfs.values() 
    for etf in subcategory
]

# Remove duplicates while preserving order
sector_etfs_list = list(OrderedDict.fromkeys(sector_etfs_list))

print(f"Total Sectors ETFs: {len(sector_etfs_list)}")  # Count of unique ETFs

Total Sectors ETFs: 93


In [12]:
ficc_etfs_list = [
    etf 
    for asset_class in ficc_etfs.values() 
    for subcategory in asset_class.values() 
    for etf in subcategory
]

# Remove duplicates while preserving order
ficc_etfs_list = list(OrderedDict.fromkeys(ficc_etfs_list))

print(f"Total FICC ETFs: {len(ficc_etfs_list)}")

Total FICC ETFs: 88


In [13]:
# Combines ficc_etfs_list and sector_etfs_list into a single list with:
# 1. All duplicates removed (keeps only first occurrence)
# 2. Original order preserved (first appearance order)
# Example: If 'GLD' exists in both lists, only the first occurrence is kept
all_etfs_list = list(OrderedDict.fromkeys(ficc_etfs_list + sector_etfs_list))
print(f"Total ETFs: {len(all_etfs_list)}")

Total ETFs: 175


In [14]:
ETFs_daily_prices = get_price_matrix(all_etfs_list, '20Y')

404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/EMFX?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=EMFX&crumb=kaZ.fbOxRJf

1 Failed download:
['EMFX']: AttributeError("'NoneType' object has no attribute 'update'")
404 Client Error: Not Found for url: https://query2.finance.yahoo.com/v10/finance/quoteSummary/PSCL?modules=financialData%2CquoteType%2CdefaultKeyStatistics%2CassetProfile%2CsummaryDetail&corsDomain=finance.yahoo.com&formatted=false&symbol=PSCL&crumb=kaZ.fbOxRJf

1 Failed download:
['PSCL']: AttributeError("'NoneType' object has no attribute 'update'")

1 Failed download:
['BITE']: YFPricesMissingError('possibly delisted; no price data found  (period=20y) (Yahoo error = "No data found, symbol may be delisted")')

1 Failed download:
['IXIS']: YFPricesMissingError('possibly delisted; no price data found  (period=20y)')


In [15]:
returns_1d = rolling_calendar_returns(ETFs_daily_prices, 1)
returns_90d = rolling_calendar_returns(ETFs_daily_prices, 90)
returns_180d = rolling_calendar_returns(ETFs_daily_prices, 180)
returns_360d = rolling_calendar_returns(ETFs_daily_prices, 360)

returns_90d.to_csv('returns_90d.csv')
returns_180d.to_csv('returns_180d.csv')
returns_360d.to_csv('returns_360d.csv')
ETFs_daily_prices.to_csv('ETFs_daily_prices.csv')
strict_elementwise_average(returns_90d, returns_180d, returns_360d).to_csv('average_returns.csv')